# Test 1 short

Sometimes, dnadiff returns diffrent value for AlignedBases that our code. 

Here, is a short example with 2 genomes. For the purpose of this investigation, I kept the refernce genome unchanged, for the query genome sequences that cause the issue were kept. 

__Step 1__: Run dnadiff and obtain the value for AlignedBases.

In [1]:
from pathlib import Path
from collections import defaultdict
import intervaltree

In [2]:
!dnadiff input/GCF_003369795.1_ASM336979v1_genomic.fna input/GCF_015244315.1_troublesome_regions.fna


Building alignments
1: PREPARING DATA
2,3: RUNNING mummer AND CREATING CLUSTERS
# reading input file "out.ntref" of length 7787609
# construct suffix tree for sequence of length 7787609
# (maximum reference length is 536870908)
# (maximum query length is 4294967295)
# process 77876 characters per dot
#....................................................................................................
# CONSTRUCTIONTIME /opt/anaconda3/envs/phd/opt/mummer-3.23/mummer out.ntref 3.60
# reading input file "/Users/angelikakiepas/Desktop/test_1_incongruencies/input/GCF_015244315.1_troublesome_regions.fna" of length 1211
# matching query-file "/Users/angelikakiepas/Desktop/test_1_incongruencies/input/GCF_015244315.1_troublesome_regions.fna"
# against subject-file "out.ntref"
# COMPLETETIME /opt/anaconda3/envs/phd/opt/mummer-3.23/mummer out.ntref 3.61
# SPACE /opt/anaconda3/envs/phd/opt/mummer-3.23/mummer out.ntref 7.55
4: FINISHING DATA
Filtering alignments
Extracting alignment coordinates
Ana

Let's see what the AlignedBases value is.

In [3]:
!cat out.report

/Users/angelikakiepas/Desktop/test_1_incongruencies/input/GCF_003369795.1_ASM336979v1_genomic.fna /Users/angelikakiepas/Desktop/test_1_incongruencies/input/GCF_015244315.1_troublesome_regions.fna
NUCMER

                               [REF]                [QRY]
[Sequences]
TotalSeqs                          1                    3
AlignedSeqs               1(100.00%)           3(100.00%)
UnalignedSeqs               0(0.00%)             0(0.00%)

[Bases]
TotalBases                   7787608                 1209
AlignedBases              716(0.01%)          968(80.07%)
UnalignedBases       7786892(99.99%)          241(19.93%)

[Alignments]
1-to-1                             2                    2
TotalLength                      716                  716
AvgLength                     358.00               358.00
AvgIdentity                    90.65                90.65

M-to-M                             3                    3
TotalLength                      965                  968
AvgLen

**Step 2:** Calculate the total number of aligned bases with interval tree

In [4]:
def parse_delta(infname):
    """Parse delta files.

    :param infname: Path to delta file
    """

    current_ref = None
    current_qry = None

    regions_ref = defaultdict(list) #Hold a dictionary for refence regions
    regions_qry = defaultdict(list) #Hold a dictionary for query regions


    for line in [_.strip().split() for _ in infname.open("r").readlines()]:

            if line[0] == "NUCMER":  # Skip headers
                    continue
            if line[0].startswith(">"):
                current_ref = line[0].strip('>')
                current_qry = line[1]
            if len(line) == 7:
                regions_ref[current_ref].append(tuple(sorted(list([int(line[0]), int(line[1])])))) #aligned regions reference
                regions_qry[current_qry].append(tuple(sorted(list([int(line[2]), int(line[3])])))) #aligned regions qry


    ref_total_aligned_size = 0
    for key in regions_ref:
        ref_tree = intervaltree.IntervalTree.from_tuples(regions_ref[key])
        ref_tree.merge_overlaps(strict=False)
        print(ref_tree)
        ref_aligned_size = 0
        for interval in ref_tree:
            ref_total_aligned_size += interval.end - interval.begin + 1
            
    return ref_total_aligned_size

In [5]:
print(parse_delta(Path("out.mdelta")))

IntervalTree([Interval(4258377, 4258701), Interval(4263071, 4263462)])
717
